In [1]:
import pandas as pd
import random
import numpy as np
from selenium.webdriver.support.ui import Select
import openpyxl
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import date
import os
import ba
import base64
import pytesseract
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import io

from selenium.common.exceptions import NoSuchElementException
import re
models = ['https://www.avito.ru/moskva/predlozheniya_uslug/delovye_uslugi/konsultirovanie-ASgBAgICAkSYC7KfAZ4L~J8B?q=%D0%B8%D0%BF%D0%BE%D1%82%D0%B5%D1%87%D0%BD%D1%8B%D0%B9+%D0%B1%D1%80%D0%BE%D0%BA%D0%B5%D1%80']
for ind, model in enumerate(models):
    file_= r"c:/games/parser/"
    file_ = file_ + 'insert' + ".xlsx"
    file = file_ + str(date.today())
    wb = openpyxl.Workbook()
    wb.save(file_)
    counter = 0
    number_start = 0
    url_start = model
    browser = webdriver.Chrome()
    browser.get(url_start)
    time.sleep(15)
    next_page='button'
    while next_page:
        soup = BeautifulSoup(browser.page_source,'html.parser')
        cards = soup.find_all('div', class_="iva-item-root-_lk9K photo-slider-slider-S15A_ iva-item-list-rfgcH iva-item-redesign-rop6P iva-item-responsive-_lbhG items-item-My3ih items-listItem-Gd1jN js-catalog-item-enum")
        for card in cards:
            counter = counter+1
            companies = []
            numbers = []
            names = []
            card_text = []
            name = card.find('h3', itemprop="name")
            names.append(name.text)
            company = card.find('a', target="_blank")
            companies.append('https://www.avito.ru'+str(company.attrs['href']))
            numbers.append('+')
            string = card.text
            card_text.append(string.replace("\n"," ")[:300])
            try:
                matrix = np.array([companies, numbers, names, card_text])
                matrix = matrix.transpose()
                df = pd.DataFrame(data=(matrix), columns=['Компания','Номер','Название','text'])
                # здесь путь к результирующему файлу
                with pd.ExcelWriter(file_, mode='a', engine="openpyxl",if_sheet_exists='overlay') as writer:
                    df.to_excel(writer, sheet_name='Sheet', startrow=counter, index=False, header=False)
            except:
                print('damn')
        try:
            time.sleep(7)
            next_page = browser.find_element(By.XPATH, '//a[@aria-label="Следующая страница"]')
            next_page.click()
        except:
            next_page = None

    time.sleep(5)
    browser.quit()

startrow = 0


In [54]:
datafr = pd.read_excel(r"c:/games/parser/insert.xlsx", names=['href', 'number', 'name', 'text'])
hrefs = datafr.href.tolist()
texts = []
names_post = []
telefones = []
names_comp = []
for index, url in enumerate(hrefs[0:100]):
    browser = webdriver.Chrome()
    browser.set_window_position(-10000,0)
    browser.get(url)
    time.sleep(5)
    try:
        phone_number = browser.find_element(By.XPATH, '//button[@data-marker="item-phone-button/card"]')
        phone_number.click()
        time.sleep(7)
    except:
        pass
    soup = BeautifulSoup(browser.page_source,'html.parser')
    phone = soup.find('img', class_="item-popup-phone-module-phone-image-DvMBv")
    comp = soup.find('a', class_="styles-module-root-QmppR styles-module-root_noVisited-aFA10 styles-module-root_preset_black-JkIdG")
    if not comp:
        comp = soup.find('h4', class_="styles-module-root-TWVKW styles-module-root-_KFFt styles-module-size_l-_oGDF styles-module-size_l_dense-Wae_G styles-module-size_l-hruVE styles-module-size_dense-z56yO stylesMarningNormal-module-root-OSCNq stylesMarningNormal-module-header-l-qvNIS")
    try:
        href_profile = comp.attrs['href']
        if href_profile not in url_comp:
            url_comp.append(comp.attrs['href'])
            fa = io.BytesIO(base64.b64decode(phone.attrs['src'].partition('base64,')[2]))
            image = Image.open(fa)
            string = pytesseract.image_to_string(image)
            names_comp.append(comp.text)
            telefones.append(string)
            names_post.append(datafr.name.iloc[hrefs.index(url)])
            texts.append(datafr.text.iloc[hrefs.index(url)].replace("\n"," ")[:300])
            number_start = number_start+1 
    except:
        try:
            fa = io.BytesIO(base64.b64decode(phone.attrs['src'].partition('base64,')[2]))
            image = Image.open(fa)
            string = pytesseract.image_to_string(image)
            if string not in telefones:
                names_comp.append(comp.text)
                telefones.append(string)
                names_post.append(datafr.name.iloc[hrefs.index(url)])
                texts.append(datafr.text.iloc[hrefs.index(url)].replace("\n"," ")[:300])
                number_start = number_start+1
        except:
            pass
    browser.quit()

file_= r"c:/games/parser/"
file_ = file_ + str(date.today()) + ".xlsx"
wb = openpyxl.Workbook()
wb.save(file_)
try:
    matrix = np.array([names_comp, telefones, names_post, texts])
    matrix = matrix.transpose()
    df = pd.DataFrame(data=(matrix), columns=['Компания','Номер','Название','Текст'])
    # здесь путь к результирующему файлу
    startrow = number_start+1
    with pd.ExcelWriter(file_, mode='a', engine="openpyxl",if_sheet_exists='overlay') as writer:
        df.to_excel(writer, sheet_name='Sheet', startrow=startrow, index=False, header=False)
    startrow = number_start+1
except:
    print('damn')
    
data_for_csv = pd.read_excel(file_, header=None)
data_for_csv.to_csv(file + str(date.today())+'.csv', encoding='utf-8', sep=';')